## Tarea 5 - Introspección | Detección de elementos específicos (Islas Rojas)
    Ensayo | Solucion de Ángel Arreola
    

#### Introducción y Contexto del Código para la Detección de Islas Rojas en Imágenes
Este código para la detección de objetos de color rojo en imágenes digitales es una evolución del algoritmo previamente desarrollado para identificar islas en imágenes. Tomando como base el enfoque y las técnicas utilizadas en el ejemplo anterior, este nuevo algoritmo se adapta y mejora para enfocarse específicamente en la detección de objetos rojos. Al igual que el código original, emplea la biblioteca OpenCV y NumPy, aprovechando el potencial de Python en el procesamiento de imágenes y la visión por computadora. Sin embargo, introduce cambios significativos en la identificación del color y el tratamiento de los datos de la imagen, reflejando un avance en la complejidad y la especificidad de la tarea de detección.

![Alt text](imagenIslasRojas.png)

### Análisis y Teoría detrás del Código para la Detección de Islas Rojas en Imágenes

El código presentado implementa un algoritmo para detectar y localizar objetos de color rojo en una imagen digital, utilizando la biblioteca OpenCV y NumPy en Python. Este algoritmo se basa en el concepto de Flood Fill y la manipulación del espacio de color HSV (Hue, Saturation, Value). A continuación, se detallan los aspectos teóricos y prácticos clave de esta solución.

#### 1. Espacio de Color HSV

- **Selección del Espacio de Color:**
  - El color de los objetos a detectar se define en el espacio HSV, que es más efectivo que el espacio RGB para la identificación de colores debido a su capacidad para separar la información de color (matiz y saturación) de la luminancia (valor).
  
- **Definición del Rojo en HSV:**
  - El rojo, al estar en los extremos del espectro de colores en HSV, se define en dos rangos: uno cerca del principio (0-10) y otro cerca del final (170-180) del eje de matiz.

#### 2. Flood Fill para la Detección de Objetos

- **Funcionamiento del Flood Fill:**
El algoritmo Flood Fill comienza en un píxel (semilla) y se expande a los píxeles vecinos, agrupando aquellos que cumplen con un criterio específico, en este caso, estar dentro del rango de color rojo definido.

- **Implementación en el Código:**
La función "flood_fill" recibe el espacio de color HSV de la imagen, un punto semilla, los rangos de color rojo, un array de píxeles visitados y un tamaño mínimo para los objetos. Recorre los píxeles vecinos, verificando si están dentro del rango de color rojo y si ya han sido visitados, agregándolos a la lista de píxeles del objeto.

#### 3. Validación de Color

- **Función "is_valid_color":**
Esta función verifica si un píxel está dentro del rango de color rojo definido en HSV. Facilita la identificación de píxeles que pertenecen a los objetos de interés.

#### 4. Localización de Objetos y Umbral de Tamaño

- **Función "find_objects":**
Convierte la imagen al espacio HSV, recorre cada píxel y aplica "flood_fill" si el píxel es rojo y no ha sido visitado. Calcula el centroide de los píxeles que forman un objeto, considerando solo aquellos con un tamaño mayor al mínimo definido.

#### 5. Procesamiento y Visualización Final

- **Detección y Dibujo de Objetos:**
El código carga la imagen, aplica "find_objects" para cada rango de rojo en HSV, dibuja círculos en los centros de los objetos detectados y muestra la imagen final.

In [1]:
import cv2
import numpy as np

def is_valid_color(hsv_pixel, lower_red, upper_red):
    """ Verifica si el color HSV está dentro del rango del rojo. """
    return all(lower_red <= hsv_pixel) and all(hsv_pixel <= upper_red)

def flood_fill(hsv, seed_point, lower_red, upper_red, visited, min_size):
    rows, cols = hsv.shape[:2]
    object_pixels = []
    queue = [seed_point]

    while queue:
        x, y = queue.pop(0)
        if visited[y, x]:
            continue

        current_color = hsv[y, x]
        if is_valid_color(current_color, lower_red, upper_red):
            visited[y, x] = True
            object_pixels.append((x, y))

            # Agregar vecinos a la cola
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < cols and 0 <= ny < rows and not visited[ny, nx]:
                    queue.append((nx, ny))

    if len(object_pixels) >= min_size:
        return object_pixels
    return []

def find_objects(image, lower_red, upper_red, min_size):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    visited = np.zeros(hsv.shape[:2], dtype=bool)
    object_centers = []

    for y in range(hsv.shape[0]):
        for x in range(hsv.shape[1]):
            if is_valid_color(hsv[y, x], lower_red, upper_red) and not visited[y, x]:
                object_pixels = flood_fill(hsv, (x, y), lower_red, upper_red, visited, min_size)
                if object_pixels:
                    object_center = np.mean(object_pixels, axis=0)
                    object_centers.append((int(object_center[0]), int(object_center[1])))

    return object_centers

# Definir el tamaño mínimo para los objetos detectados (por ejemplo, 30 píxeles)
min_size = 20

# Definir los rangos de rojo en HSV
lower_red1 = np.array([0, 150, 50])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 150, 50])
upper_red2 = np.array([180, 255, 255])


# Cargar la imagen
image_path = 'C:\\Users\\angel\\Desktop\\AI\\Tareas\\imagenIslasRojas.png'
image = cv2.imread(image_path)

# Encontrar los objetos rojos utilizando flood fill
object_centers = find_objects(image, lower_red1, upper_red1, min_size)
object_centers.extend(find_objects(image, lower_red2, upper_red2, min_size))

# Mostrar las coordenadas de los centros de los objetos rojos detectados
for idx, center in enumerate(object_centers):
    print(f"Centro del objeto rojo {idx+1}: {center}")

# Dibujar círculos amarillos en los centros de los objetos rojos
for center in object_centers:
    cv2.circle(image, center=center, radius=5, color=(0, 255, 255), thickness=2)

# Guardar y mostrar la imagen resultante
output_image_path = 'C:\\Users\\angel\\Desktop\\AI\\Tareas\\detected_red_objects.png'
cv2.imwrite(output_image_path, image)
cv2.imshow('Detected Red Objects', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Centro del objeto rojo 1: (446, 16)
Centro del objeto rojo 2: (582, 26)
Centro del objeto rojo 3: (604, 22)
Centro del objeto rojo 4: (557, 27)
Centro del objeto rojo 5: (442, 25)
Centro del objeto rojo 6: (128, 43)
Centro del objeto rojo 7: (24, 33)
Centro del objeto rojo 8: (309, 36)
Centro del objeto rojo 9: (251, 39)
Centro del objeto rojo 10: (261, 47)
Centro del objeto rojo 11: (177, 56)
Centro del objeto rojo 12: (151, 59)
Centro del objeto rojo 13: (214, 54)
Centro del objeto rojo 14: (260, 58)
Centro del objeto rojo 15: (200, 60)
Centro del objeto rojo 16: (167, 73)
Centro del objeto rojo 17: (287, 65)
Centro del objeto rojo 18: (107, 65)
Centro del objeto rojo 19: (194, 67)
Centro del objeto rojo 20: (338, 66)
Centro del objeto rojo 21: (135, 70)
Centro del objeto rojo 22: (105, 71)
Centro del objeto rojo 23: (286, 75)
Centro del objeto rojo 24: (126, 80)
Centro del objeto rojo 25: (208, 80)
Centro del objeto rojo 26: (142, 83)
Centro del objeto rojo 27: (245, 109)
Centro del

#### Conclusión
Este código ilustra una aplicación efectiva de Flood Fill y la manipulación del espacio de color HSV para la detección de objetos específicos en imágenes. La elección de HSV para la definición del color y el uso de Flood Fill para la agrupación de píxeles ofrecen una metodología robusta y precisa para identificar y localizar objetos de color rojo en una imagen. Este enfoque demuestra cómo los conceptos de la visión por computadora y la ciencia de la computación pueden combinarse para resolver problemas complejos de procesamiento de imágenes.

![Alt text](detected_red_objects%20copy.png)

### Version usando Find Contours

In [2]:
import cv2
import numpy as np

def find_red_objects(image):
    # Convertir la imagen a espacio de color HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Ajustar los rangos de color rojo para capturar solo los tonos rojos
    # HSV values: [Hue, Saturation, Value]
    lower_red1 = np.array([0, 150, 50])  # Ajustar la saturación y el valor para ser más específico para el rojo
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 150, 50])
    upper_red2 = np.array([180, 255, 255])

    # Crear dos máscaras para el rango de rojo y luego combinarlas
    mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Usar dilatación para conectar componentes rojos
    kernel = np.ones((3,3), np.uint8)
    red_mask_dilated = cv2.dilate(red_mask, kernel, iterations=2)

    # Encontrar contornos en la máscara roja dilatada
    contours, _ = cv2.findContours(red_mask_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convertir la imagen a escala de grises para el fondo
    grey_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    grey_image_colored = cv2.cvtColor(grey_image, cv2.COLOR_GRAY2BGR)

    # Pintar los objetos rojos en la imagen en escala de grises
    colored_objects_mask = np.zeros_like(image)
    cv2.drawContours(colored_objects_mask, contours, -1, (255, 255, 255), cv2.FILLED)
    grey_image_colored[colored_objects_mask == 255] = image[colored_objects_mask == 255]

    red_objects_centers = []

    # Calcular el centroide de cada contorno rojo detectado y dibujar un círculo amarillo
    for contour in contours:
        M = cv2.moments(contour)
        if M["m00"] > 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            red_objects_centers.append((cX, cY))
            cv2.circle(grey_image_colored, (cX, cY), 7, (0, 255, 255), -1)  # Amarillo

    return red_objects_centers, grey_image_colored

# Cargar la imagen
image_path = 'C:\\Users\\angel\\Desktop\\AI\\Tareas\\imagenIslasRojas.png'
image = cv2.imread(image_path)

# Encontrar los objetos rojos y mostrarlos en su color original sobre fondo gris
red_objects_centers, result_image = find_red_objects(image)

# Imprimir las coordenadas de los centros de los objetos rojos detectados
for idx, center in enumerate(red_objects_centers):
    print(f"Centro del objeto rojo {idx+1}: {center}")

# Guardar la imagen resultante
output_image_path = 'C:\\Users\\angel\\Desktop\\AI\\Tareas\\detected_red_objects_on_grey.png'
cv2.imwrite(output_image_path, result_image)

# Mostrar la imagen resultante con los objetos rojos marcados
cv2.imshow('Red Objects on Grey', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

output_image_path


Centro del objeto rojo 1: (581, 624)
Centro del objeto rojo 2: (328, 619)
Centro del objeto rojo 3: (213, 617)
Centro del objeto rojo 4: (607, 621)
Centro del objeto rojo 5: (260, 615)
Centro del objeto rojo 6: (223, 615)
Centro del objeto rojo 7: (230, 613)
Centro del objeto rojo 8: (174, 615)
Centro del objeto rojo 9: (508, 608)
Centro del objeto rojo 10: (482, 606)
Centro del objeto rojo 11: (245, 608)
Centro del objeto rojo 12: (368, 609)
Centro del objeto rojo 13: (349, 608)
Centro del objeto rojo 14: (386, 602)
Centro del objeto rojo 15: (474, 603)
Centro del objeto rojo 16: (231, 597)
Centro del objeto rojo 17: (502, 591)
Centro del objeto rojo 18: (491, 591)
Centro del objeto rojo 19: (430, 609)
Centro del objeto rojo 20: (276, 588)
Centro del objeto rojo 21: (351, 586)
Centro del objeto rojo 22: (335, 588)
Centro del objeto rojo 23: (499, 583)
Centro del objeto rojo 24: (476, 584)
Centro del objeto rojo 25: (496, 576)
Centro del objeto rojo 26: (234, 583)
Centro del objeto roj

'C:\\Users\\angel\\Desktop\\AI\\Tareas\\detected_red_objects_on_grey.png'

![Alt text](detected_red_objects_on_grey.png)